In [8]:
from selenium import webdriver
from datetime import date, datetime, timedelta
import time
import os
import pandas as pd

from IPython.display import display, HTML

# constant to establish download folder path, only need to change this to change location
DOWNLOAD_FOLDER = r"C:\Users\boa.sokchu\Downloads"

# list of files before downloading
before = os.listdir(DOWNLOAD_FOLDER)

# activate chrome driver
browser = webdriver.Chrome()
browser.maximize_window()
browser.get("https://boa.3plsystemscloud.com/")

# page elements to login
boa_user = browser.find_element_by_id("txb-username")
boa_pw = browser.find_element_by_id("txb-password")
login_button = browser.find_element_by_id("ctl00_ContentBody_butLogin")

# login credentials
boa_user.send_keys("sokchu@boalogistics.com")
boa_pw.send_keys("Immuneact10!")
login_button.click()

# enter report code into report_code variable
# "Local Bobtail Loads" report
report_code = "70E72373FD1D"
url = "https://boa.3plsystemscloud.com/App_BW/staff/Reports/ReportViewer.aspx?code="+report_code
browser.get(url)

# sets up start date and end date for filter
# checks today's date to calculate next 2 business days

from datetime import datetime, timedelta
now = datetime.now()
monday = now - timedelta(days = now.weekday())
print(monday)

add_day = 6

s_date = monday
e_date = monday + timedelta(add_day)
start = s_date.strftime("%m/%d/%Y 00:00:00")
end = e_date.strftime("%m/%d/%Y 23:59:59")

# combining string conversion to 1 line each, ok to delete 2 lines below if working properly
# start = str_s_date + " 00:00:00"
# end = str_e_date + " 23:59:59"

# set up variables for parameter fields
startbox = browser.find_element_by_xpath("//td[5]/input[@class='filter between'][1]")
endbox = browser.find_element_by_xpath("//td[5]/input[@class='filter between'][2]")

# inserts new parameters
startbox.clear()
startbox.send_keys(start)
endbox.clear()
endbox.send_keys(end)

# save & view report, then download
save_button = browser.find_element_by_id("ctl00_ContentBody_butSaveView").click()
browser.implicitly_wait(3)
download = browser.find_element_by_id("ctl00_ContentBody_butExportToExcel").click()

#need to wait a few seconds before continuing to allow for file to finish downloading.

time.sleep(2)

browser.quit()


#compares list of files in Downloads folder after downloading file to extract filename
after = os.listdir(DOWNLOAD_FOLDER)
change = set(after) - set(before)

if len(change) == 1:
    file_name = change.pop()
    print(file_name + " downloaded.")
else:
    print ("More than one file or no file downloaded")
    
# sets filepath to downloaded file and create DataFrame from file 
# *output file extension is .xls but is actually.html format
filepath = DOWNLOAD_FOLDER + "\\" + file_name
data = pd.read_html(filepath)
df = data[0]
df.fillna('',inplace=True)


# gets today's date and adds it to new file name then saves as .xlsx file

output_path = DOWNLOAD_FOLDER + "\\" + file_name[0:len(file_name)-4] + ".xlsx"

writer = pd.ExcelWriter(output_path, engine="xlsxwriter")
df.to_excel(writer, sheet_name='DAILYREPORT')
writer.save()
print("Saved file to " + output_path + "!")

2021-07-19 13:28:43.316839
REEFERTMSWeeklyLoadReport20210719 (4).xls downloaded.
Saved file to C:\Users\boa.sokchu\Downloads\REEFERTMSWeeklyLoadReport20210719 (4).xlsx!
